A good first step when working with text is to split it into words.
Words are called tokens and the process of splitting text into tokens is called tokenization.
Keras provides the text_to_word_sequence() function that you can use to split text into a list of words
By default, this function automatically does 3 things:
    Splits words by space (split=” “).
    Filters out punctuation (filters=’!”#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n’).
    Converts text to lowercase (lower=True)

In [4]:
from keras.preprocessing.text import text_to_word_sequence
text = 'the quick brown fox jumped over the lazy dog'
# tokenize this document
result = text_to_word_sequence(text)
print(result)

['the', 'quick', 'brown', 'fox', 'jumped', 'over', 'the', 'lazy', 'dog']


Encoding with one_hot
It is popular to represent a document as a sequence of integer values, where each word in the 
document is represented as a unique integer
Keras provides the one_hot() function that you can use to tokenize and integer encode a text 
document in one step. The name suggests that it will create a one-hot encoding of the document, 
which is NOT the case.

Instead, the function is a wrapper for the hashing_trick() function. The function returns an 
integer encoded version of the document. The use of a hash function means that there may be 
collisions and not all words will be assigned unique integer values.

As with the text_to_word_sequence() function in the previous section, the one_hot() function will 
make the text lower case, filter out punctuation, and split words based on white space.


In [6]:
from keras.preprocessing.text import one_hot, text_to_word_sequence
text = 'The quick brown fox jumped over the lazy dog.'
# estimate the size of vocabulary
words = set(text_to_word_sequence(text))
vocab_size = len(words)
print(vocab_size)
# integer encode the document
# The size of the vocabulary defines the hashing space from which words are hashed. Ideally, this 
# should be larger than the vocabulary by some percentage (perhaps 25%) to minimize the number of 
# collisions
result = one_hot(text, round(vocab_size*1.3))
print(result)


8
[9, 2, 7, 6, 7, 5, 9, 8, 9]


A limitation of integer and count base encodings is that they must maintain a vocabulary of words 
and their mapping to integers.

An alternative to this approach is to use a one-way hash function to convert words to integers. 
This avoids the need to keep track of a vocabulary, which is faster and requires less memory.

Keras provides the hashing_trick() function that tokenizes and then integer encodes the document,
just like the one_hot() function. It provides more flexibility, allowing you to specify the hash 
function as either ‘hash’ (the default) or other hash functions such as the built in md5 function 
or your own function.

In [7]:
from keras.preprocessing.text import hashing_trick, text_to_word_sequence
text = 'The quick brown fox jumped over the lazy dog.'
# estimate the size of the vocabulary
words = set(text_to_word_sequence(text))
vocab_size = len(words)
print(vocab_size)
# integer encode the document
result = hashing_trick(text, round(vocab_size*1.3), hash_function='md5')
print(result)
# We can see that the use of a different hash function results in consistent, but different 
# integers for words as the one_hot() function in the previous section

8
[6, 4, 1, 2, 7, 5, 6, 2, 6]


Keras provides a more sophisticated API for preparing text that can be fit and reused to prepare 
multiple text documents. This may be the preferred approach for large projects.

Keras provides the Tokenizer class for preparing text documents for deep learning. The Tokenizer
must be constructed and then fit on either raw text documents or integer encoded text documents

Once fit, the Tokenizer provides 4 attributes that you can use to query what has been learned about
your documents:
    
    word_counts: A dictionary of words and their counts.
    word_docs: A dictionary of words and how many documents each appeared in.
    word_index: A dictionary of words and their uniquely assigned integers.
    document_count:An integer count of the total number of documents that were used to fit the Tokenizer.

Once the Tokenizer has been fit on training data, it can be used to encode documents in the 
train or test datasets.
The texts_to_matrix() function on the Tokenizer can be used to create one vector per document 
provided per input. The length of the vectors is the total size of the vocabulary
This function provides a suite of standard bag-of-words model text encoding schemes that can be 
provided via a mode argument to the function.

The modes available include:

    ‘binary‘: Whether or not each word is present in the document. This is the default.
    ‘count‘: The count of each word in the document.
    ‘tfidf‘: The Text Frequency-Inverse DocumentFrequency (TF-IDF) scoring for 
    each word in the  document.
    ‘freq‘: The frequency of each word as a ratio of words within each document.


In [8]:
from keras.preprocessing.text import Tokenizer
# define 5 documents
docs = ['Well done!','Good work','Great effort','nice work','Excellent!']

# create the tokenizer
t = Tokenizer()
# fit the tokenizer on the documents
t.fit_on_texts(docs)

In [12]:
# summarize what was learned
print(t.word_counts)
print(t.document_count)
print(t.word_index)
print(t.word_docs)

OrderedDict([('well', 1), ('done', 1), ('good', 1), ('work', 2), ('great', 1), ('effort', 1), ('nice', 1), ('excellent', 1)])
5
{'work': 1, 'well': 2, 'done': 3, 'good': 4, 'great': 5, 'effort': 6, 'nice': 7, 'excellent': 8}
defaultdict(<class 'int'>, {'well': 1, 'done': 1, 'work': 2, 'good': 1, 'great': 1, 'effort': 1, 'nice': 1, 'excellent': 1})


In [13]:
# define 5 documents
docs = ['Well done!','Good work','Great effort','nice work','Excellent!']
# create the tokenizer
t = Tokenizer()
# fit the tokenizer on the documents
t.fit_on_texts(docs)
# summarize what was learned
print(t.word_counts)
print(t.document_count)
print(t.word_index)
print(t.word_docs)
# integer encode documents
encoded_docs = t.texts_to_matrix(docs, mode='count')
print(encoded_docs)

OrderedDict([('well', 1), ('done', 1), ('good', 1), ('work', 2), ('great', 1), ('effort', 1), ('nice', 1), ('excellent', 1)])
5
{'work': 1, 'well': 2, 'done': 3, 'good': 4, 'great': 5, 'effort': 6, 'nice': 7, 'excellent': 8}
defaultdict(<class 'int'>, {'well': 1, 'done': 1, 'work': 2, 'good': 1, 'great': 1, 'effort': 1, 'nice': 1, 'excellent': 1})
[[0. 0. 1. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]]
